# **TEAM ALLOCATION SIMULATOR - FDAE Group 4** #

Team members information:  
Nguyen Xuan Hoang - hoang013@e.ntu.edu.sg\
Lai SiZhu - la0001hu@e.ntu.edu.sg\
Guntaphon Akwatanakun gunt0005@e.ntu.edu.sg\
Alicia Ong (Wang Huimin) - alic0034@e.ntu.edu.sg

## **Overview of the problem:**
The problem requires us to:
- Analyse the `records.csv` file.
- Create an algorithm to allocate students and return a `.csv` file with the student's allocated groups.
- Evaluate our algorithm.

We decided to divide the problem into 6 subparts:
1. Read `records.csv` and return the student's information as an analysable data structure.
2. Analyse the `records.csv` file.
3. Make functions to allocate the students into groups.
4. Make functions to changing some student's groups to further enhance diversity.
5. Write the output file.
6. Analyse our output `.csv` file.


### **1. Read `records.csv` and return the student's information as an analysable data structure:**

We will read the file and convert each line (each student's information) into a dictionary*. The output will be a dictionary with the `key:value` pair as their tutorial group number and a list of all members with the dictionary that has the student's info inside that. We will do with 4 steps:

1. Read `records.csv`.
2. Convert each line into a dictionary.
3. Gather the dictionary from 2. into one big list.
4. Rearrange the list from 3. into an output file as specified above.

###### *The reason why we contain every student's information as a dictionary, not a tuple is because it is easier to add in their allocated group's information with the existing student's information as a `key:value` pair.

This is the **pseudocode** and **flowchart** of this part:

**Pseudocode:**

*START*

*OPEN* file "record.csv" \
&nbsp;&nbsp;&nbsp;&nbsp;*READ* 1 line to skip header \
&nbsp;&nbsp;&nbsp;&nbsp;*SET* empty list called "students" 

*FOR* line in file \
&nbsp;&nbsp;&nbsp;&nbsp;*READ* 1 line \
&nbsp;&nbsp;&nbsp;&nbsp;*CONVERT* line into dictionary that contain student's information \
&nbsp;&nbsp;&nbsp;&nbsp;*APPEND* dictionary into students \
*ENDFOR* 

*SET* empty dictionary called "groups" 

*FOR* stu in students \
&nbsp;&nbsp;&nbsp;&nbsp;*GET* tutorial_group from stu \
&nbsp;&nbsp;&nbsp;&nbsp;*IF* tutorial_group not in groups THEN \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*CREATE* key: tutorial_group, value: [] \
&nbsp;&nbsp;&nbsp;&nbsp;*ENDIF* \
&nbsp;&nbsp;&nbsp;&nbsp;*APPEND* stu into groups["tutorial_group"] \
*ENDFOR* 

*RETURN* groups 

*END* 

----------------------------------------------------------------------------------------------------------------
**Flowchart:**
 #Fix!!!. \

----------------------------------------------------------------------------------------------------------------

The code below is the **Python code** for part **1.**:

In [491]:
#Function to read the files.
"""
convert each line into dictionary like:
{
     "tutorial_group": "G-1",
     "student_id": "U1234567X",
     "school": "SCSE",
     "name": "Zone",
     "gender": "M",
     "cgpa": 4.23
   }
"""
#function convert each line to dictionary
def ConvertToDict(line):
    #split line into list with ,
    lists = line.strip().split(",")

    #return dictionary
    return{
        "tutorial_group" : lists[0],
        "student_id" : lists[1],
        "school" : lists[2],
        "name" : lists[3],
        "gender" : lists[4],
        "cgpa" : float(lists[5])
    }

#function append each dictionary to 1 big list
def OneBigList(data = "records.csv"):
    #create 1 big list
    students = []

    #read each line from file
    with open(data, "r", encoding="utf-8") as file:
        #skip header
        header = file.readline().strip()

        #read each line and append to students
        for line in file:
            line = line.strip()
            students.append(ConvertToDict(line))
    return students

#function to gather same tut group in 1 dictionary with key = tut no.
def GroupByTut(students):
    #create dictionary
    groups = {}

    #d is dictionary
    for d in students:
        #assign tut no.
        tutno = d["tutorial_group"]

        #check tutno in groups
        if tutno not in groups:
            #create key in groups
            groups[tutno] = []
        
        #append data into groups(list)
        groups[tutno].append(d)

    #return big dictionary
    return groups

#comebine functions
def readfile():
    students = OneBigList("records.csv")
    groups = GroupByTut(students)
    return groups

### **3. Allocate the students within a tutorial group into subgroups based on gender and CGPA diversity.**

Because allocating students into subgroups to ensure school, gender, and CGPA diversity is very complex, in this step we will only allocate students into subgroups of 5 students to ensure **gender** and **CGPA** diversity, then we will swap students in different groups to ensure school diversity later

We will do this in **9** steps:

**Step 1:** Split the list of students in a tutorial group into 2 groups containing only Male and Female students, then create 2 new male and female lists with their **CGPA** in descending order.

**Step 2:** Calculate the ratio of male:female students in a tutorial group. We calculate this ratio to find out about any tutorial group that is too skewered towards one gender. For groups of 5 students, the most gender-diverse type of group is 2 male - 3 female or 3 male - 2 female. So in case one gender in the tutorial group does not have at least **20** people, we must accept some groups with 1 male - 4 female or 4 male - 1 female. This corresponds to the ratio of male:female students to be less than **2/3** or larger than **3/2**.

**Step 3:** After finding the ratio of male:female students in a tutorial group, if the ratio is less than **2/3** or larger than **3/2**, we will find the amount of groups that are 1 male - 4 female if the male:female ratio is less than **2/3** or the number of groups that are 4 male - 1 female if the male:female ratio is more than **3/2**. We do this by solving the matrix equation below:
$$
    \begin{bmatrix}
    2 & 1 \\
    3 & 4
    \end{bmatrix}
    \times
    \begin{bmatrix}
    m \\
    n
    \end{bmatrix}
    =
    \begin{bmatrix}
    x \\
    y
    \end{bmatrix}
$$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In the equation above:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - $m$ is the number of groups with a 3-2 member composition. \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - $n$ is the number of groups with a 4-1 member composition. \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - $x$ is the number of members having the minority gender. \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - $y$ is the number of members having the majority gender.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Solving the matrix equation gives us $m = \frac{4x - y}{5}$, $n = \frac{-3x + 2y}{5}$. \
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Combining with $m + n = 10$ and $x + y = 50$ gives us:
$$
    m = x - 10 \\
    n = 20 - x
$$

**Step 4:** Create 10 groups of 4.

&nbsp;&nbsp;&nbsp;&nbsp;We pick four students first. There will be **3** types of 4-people groups in our sorting:

&nbsp;&nbsp;&nbsp;&nbsp;- 2 male - 2 female (for any type of group) \
&nbsp;&nbsp;&nbsp;&nbsp;- 1 male - 3 female (for female-majority tutorial groups) \
&nbsp;&nbsp;&nbsp;&nbsp;- 3 male - 1 female (for male-majority tutorial groups) 

&nbsp;&nbsp;&nbsp;&nbsp;We can ensure that these groups of 4 have a mix of high cgpa and low cgpa by popping from the beginning and the end of the sorted male and &nbsp;&nbsp;&nbsp;&nbsp;female lists. 

&nbsp;&nbsp;&nbsp;&nbsp;The reason why we do this is by the time we have made 10 groups, we can allocate the remaining students into groups so that the gender diversity &nbsp;&nbsp;&nbsp;&nbsp;is most optimised (i.e. the last students will make the group gender dynamics 2-3 or 1-4)

**Step 5:** Create an unassigned students list of the unassigned male and female students by merging the sorted male and female list together. Then we create a new list that sorts the students by descending CGPA.

**Step 6:** Sort the allocated 4 people groups in ascending order by the group's average CGPA.

**Step 7:** Allocate the final members of the group into the groups of 4 by rainbow-matching the 4 people group with the highest average CGPA and the student with lowest CGPA in the unassigned student list.

**Step 8:** In case of any student is still unassigned and still remains in the unassigned list (although it should be empty), we make them into one final group and append it into our list of subgroups. 

**Step 9:** Return the allocated groups.

The code below is the **Python code** for part **3.**:

In [492]:
# Function to allocate students in a tutorial group into groups of 5 students, based on gender and CGPA.
def Groups_of_5(student_list):

    #Helper functions to help sort students by and the 4 people groups in Step 1 and Step 6.
    #Code to sort by CGPA of a group (Used in Step 1):
    def criteria_cgpa(student):
        return student["cgpa"]

    def SortbyCGPA(student_list, reverse_or_not):
        return sorted(student_list, reverse=reverse_or_not, key=criteria_cgpa)

    #Code to sort the 4 people groups by ascending CGPA (Used in Step 6):
    def mean_cgpa(team):
        total = 0
        for s in team:
            total += s["cgpa"]
        return total / len(team)

    def SortSubgroupsByMeanCGPA(all_subgroups):
        return sorted(all_subgroups, key=mean_cgpa)
    #_____________________________________________________________________#

    #Code to classify Male and Female students and sort them by ascending/descending GPA (Step 1):
    Male_Students = [students for students in student_list if students["gender"] == "Male"]
    Female_Students = [students for students in student_list if students["gender"] == "Female"]

    number_of_male = len(Male_Students) # Number of boys in the tutorial group
    number_of_female = len(Female_Students) # Number of girls in the tutorial group

    Male_Students_sorted = SortbyCGPA(Male_Students, True) # Sort by CGPA from high -> low
    Female_Students_sorted = SortbyCGPA(Female_Students,True) # Sort by CGPA from high -> low

    # Calculate the male:female ratio as a variable: ratio_boy_girl in Step 2:
    # The variable ratio_boy_girl is used to handle skewed-gendered groups, as specified in Step 2 and Step 3.
    ratio_boy_girl = number_of_male/number_of_female
    #_____________________________________________________________________#

    #Code to allocate students into groups of 5 as specified from Step 4-8.
    all_subgroups = [] # Initialise a list to contain the allocated groups
    group_counter = 0 # Count the number of groups we have already made
    max_groups = 10 # The maximum amount of 5 people groups

    if 2/3 <= ratio_boy_girl <= 3/2: #Happens when the gender dynamics in the tutorial groups is not skewed.
        while len(Male_Students_sorted) >= 2 and len(Female_Students_sorted) >= 2 and group_counter < max_groups:
            # Make groups of 4 students as specified in Step 4.
            team = [
                    Male_Students_sorted.pop(0), #Male/High CGPA
                    Male_Students_sorted.pop(), #Male/Low CGPA
                    Female_Students_sorted.pop(0), #Female/High CGPA
                    Female_Students_sorted.pop() #Female/Low CGPA
                    ]

            all_subgroups.append(team)
            group_counter += 1


    elif ratio_boy_girl < 2/3: #Happens when the gender dynamics in the tutorial groups is female-skewed.
        while len(Male_Students_sorted) >= 2 and len(Female_Students_sorted) >= 3 and group_counter < max_groups:
            if group_counter < number_of_male - 10: # As calculated in Step 3.
                # Make groups of 4 students as specified in Step 4.
                team = [
                        Male_Students_sorted.pop(0), #Male/High CGPA
                        Male_Students_sorted.pop(), #Male/Low CGPA
                        Female_Students_sorted.pop(0), #Female/High CGPA
                        Female_Students_sorted.pop(), #Female/Low CGPA
                        ]

            elif group_counter >= number_of_male - 10:
                # Make groups of 4 students as specified in Step 4.
                team = [
                        Male_Students_sorted.pop(0), #Male/High CGPA
                        Female_Students_sorted.pop(), #Female/Low CGPA
                        Female_Students_sorted.pop(0), #Female/High CGPA
                        Female_Students_sorted.pop(), #Female/Low CGPA
                        ]

            all_subgroups.append(team)
            group_counter += 1
    
    elif ratio_boy_girl > 3/2: #Happens when the gender dynamics in the tutorial groups is male-skewed.
        while len(Male_Students_sorted) >= 3 and len(Female_Students_sorted) >= 2 and group_counter < max_groups:
            if group_counter < number_of_female - 10: # As calculated in Step 3.
                # Make groups of 4 students as specified in Step 4.
                team = [
                        Female_Students_sorted.pop(0), #Female/High CGPA
                        Female_Students_sorted.pop(), #Female/Low CGPA
                        Male_Students_sorted.pop(0), #Male/High CGPA
                        Male_Students_sorted.pop(), #Male/Low CGPA
                        ]

            elif group_counter >= number_of_female - 10:
                # Make groups of 4 students as specified in Step 4.
                team = [
                        Female_Students_sorted.pop(0), #Female/High CGPA
                        Male_Students_sorted.pop(), #Male/Low CGPA
                        Male_Students_sorted.pop(0), #Male/High CGPA
                        Male_Students_sorted.pop(), #Male/Low CGPA
                        ]

            all_subgroups.append(team)
            group_counter += 1

    #Group all remaining students into a list, then sort them with descending CGPA. (Step 5)
    Unsorted_Students = Male_Students_sorted + Female_Students_sorted
    Unsorted_Students_sorted = SortbyCGPA(Unsorted_Students, True) #Sort the remaining student's CGPA from high to low

    all_subgroups_sorted = SortSubgroupsByMeanCGPA(all_subgroups) #Sort teams' CGPA from low to high (Step 6)
    
    num_of_4_people_groups = len(all_subgroups_sorted)
    num_of_remaining_students = len(Unsorted_Students)

    for i in range(min(num_of_4_people_groups, num_of_remaining_students)): 
        if len(all_subgroups_sorted[i]) == 4: #To ensure that subgroups only have 4 students.
            all_subgroups_sorted[i].append(Unsorted_Students_sorted.pop(0)) #Rainbow-matching the students (Step 7)

    #To handle anyone remaining in the unsorted students by putting them into a group. (Step 8)
    if Unsorted_Students_sorted:
        remaining_groups = [remaining_students for remaining_students in Unsorted_Students_sorted]
        all_subgroups.append(remaining_groups)

    return all_subgroups #Step 9

### **4. Re-allocating the groups to enhance school diversity:**

Since in the previous part we have not considered school as an allocating criteria, there might be some groups that has a school majority (i.e 3+ students in the same school in their allocated group). Thus, we will be swapping students in different allocated groups (within the same tutorial group) together so that the allocated groups are more school-diversified. As we have successfully allocated students based on gender and CGPA in the previous part, we must swap students carefully to not disturb our gender and CGPA diversity. So, our logic to facilite a swap between 2 students is as follows:

Let us have two groups that we wish to swap students together called group A and group B, and the students that are swapped in their respective groups called student A and student B. The criteria to swap students is as follow:
1. Student A must not come from the same school as the students in group B.
2. Student B must not come from the same school as the students in group A.
3. Student A and B must have the same gender.
4. Student A and B must only have a CGPA difference of less than 7.5%.

This part will have **2** main parts: 
+ Part 4.1: The helper functions to assist with the swapping 
+ Part 4.2: The main function to swap the students within a group

#### **4.1. Helper functions:**

To assist in allocating the students, we have to create a few helper functions to: 
- Find the type of schools in a group.
- Check if the group is not school-diverse enough (more than `n // 3` students from same school in a group of `n` students).
- Find the first school that has more students than our school-diverse criteria in our non-school-diverse group.
- Pick the student that needs to be swapped in the non-school-diverse group.
- Check for CGPA difference between the student that needs to be swapped and potential students being picked for swapping.
- Swapping function to swap the 2 students from different allocated groups together.

We will go deeper into the details of the helper functions below.

##### **4.1.1. Find the type of schools in a group.**

We use a dictionary to count the number of school types in a group. The first loop will go through every single school and initialise the `key:value` pairs to be `<School type>:0` (E.g: `CCDS:0`). The second loop will count the number of students in that particular school.

In [493]:
def Check_School_types_in_subgroup(a_list):
    #Function to count number of school types + number of students at what school.
    Reps = {}
    for students in a_list:
        Reps[students["school"]] = 0
    for students in a_list:
        Reps[students["school"]] += 1
    return Reps

Example: A group of students have these types of schools: 

    `[{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]`

Then what the above function will return is:

    `{'CCDS': 2, 'EEE': 2, 'ASE': 1}`

In [494]:
print(Check_School_types_in_subgroup([{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]))

{'CCDS': 2, 'EEE': 2, 'ASE': 1}


##### **4.1.2. Check if the group is school-diverse enough or not.**

We called the dictionary we made in the last function and go through every single key of the dictionary. If the value associated with that key is more than the number of students in the group floor divided by 3, than that group is not school-diverse enough and we return `False`. If all values are smaller than the number of students in the group floor divided by 3, we return `True`.

In [495]:
def Check_School_Diversity(a_list):
    #Function to ensure the group has school diversity.
    School_type = Check_School_types_in_subgroup(a_list)
    number_of_students = len(a_list)
    for schools in School_type.keys():
        if School_type[schools] > number_of_students // 3:
            return False
    else:
        return True

Example: A group of students have these types of schools: 

    `[{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]`

Then what the above function will return is:

    `False`

In [496]:
print(Check_School_Diversity([{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]))

False


##### **4.1.3. Find the first school that has more students than our school-diverse criteria in our non-school-diverse group.**

We want this function to find the first type of school that is causing the group to be not diverse enough. We run through the dictionary we created with the `Check_School_types_in_subgroup` function again after finding our undesired group to find out and return the first type of school that exceeds our school diversity criteria.

In [497]:
def Check_Repeating_School(a_list):
    #Function to return the first type of school that repeats itself
    School_type = Check_School_types_in_subgroup(a_list)
    number_of_students = len(a_list)
    for schools in School_type.keys():
        if School_type[schools] > number_of_students // 3:
            return schools

Example: A group of students have these types of schools: 

    `[{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]`

Then what the above function will return is:

    `CCDS`

In [498]:
print(Check_Repeating_School([{"school":"CCDS"},{"school":"CCDS"},{"school":"EEE"},{"school":"EEE"},{"school":"ASE"}]))

CCDS


##### **4.1.4. Pick the student that needs to be swapped in the non-school-diverse group.**

We now pick the first student to swap. That student must come from the allocated group that is not school-diverse enough, and that student must also have the same school as the school that is causing the allocated group to be non-diverse. We run through the group and pick the first student that satifies the above criteria, and return his/her index in the group and his/her information.

In case if there are actually no groups that is not school diverse, then the `repeated_school` variable in the function below will be `None`, and no student comes from `None`, so we return `False` at the end of the loop.

In [499]:
def Pick_out_the_student(a_list):
    #Function to pick one student that is from a repeated school by their index in the list and themself.
    repeated_school = Check_Repeating_School(a_list)
    for student_index, student in enumerate(a_list):
        if student["school"] == repeated_school:
            return (student_index, student)
    else:
        return False

##### **4.1.5. Check for CGPA difference between the student that needs to be swapped and potential students being picked for swapping.**

After picking our first student to swap, we now pick the second student. Other than coming from a different school than the schools in the swapping group and having the same gender as the first student, we must ensure that his/her grade is within an acceptable range compared to the first student (7.5%*). If the second student's CGPA lies within 7.5% of the first student's CGPA, we return `True`, otherwise we return `False`.

In [500]:
def CGPA_variance_check(student_a, student_b):
    #Function to check for grade variance between the students
    if abs(student_a["cgpa"] - student_b["cgpa"]) <= student_a["cgpa"] * 0.075:
        return True
    else:
        return False

###### *7.5% is 0.3/4, so even the highest CGPA students (4.5+) can swap with students that are 0.34 CGPA points lower (4.2 CGPA, around median). Doing this will not affect the CGPA balance very much, since if we swap a 4.5 CGPA student and a 4.2 CGPA student, the 2 group's average CGPA will only shift by 0.06 CGPA points

##### **4.1.6. Swapping function to swap the 2 students from different allocated groups together.**

We call the tutorial group that has its members allocated into subgroups in part 2, the indices of the 2 subgroups that has students that can be swapped in the two groups, as well as the indices of the 2 students those 2 subgroups. After swapping, we return the tutorial group with the two students being swapped put into new subgroups.

In [501]:
def Group_swapper(list_of_lists, group_a, student_a, group_b, student_b):
    #Function to swap students using their indices in the big list
    list_of_lists[group_b][student_b], list_of_lists[group_a][student_a] = list_of_lists[group_a][student_a], list_of_lists[group_b][student_b]
    return list_of_lists

#### **4.2. Main function:**

The main function that will facilitate swapping the students. It will keep swapping the students until either one of these two conditions happen (`break` conditions):
1. Every group after swapping is school diverse.
2. We cannot swap anymore. This happens in cases when the number of students coming from the same school in a tutorial group is larger than (the number of subgroups) * (the number of students in a subgroup // 3), or we cannot find a suitable candidate to swap.

In one iteration of the main `while` loop, there are **4** main steps:

**Step 1:** We will set the index and value of the non-school diverse groups as sentinel values (-1 and [ ] respectively).

**Step 2:** We go through every subgroup and every student to find the first non-school-diverse group, as well as the student we wish to swap by using the `Check_School_Diversity` function (Function **4.1.2**). When we found the first non-school-diverse group, we save its index and the group itself. We also call the `Check_School_types_in_subgroup` function (Function **4.1.1**) on our newly saved group to find out about the schools types in the group. This is to check conditions 1 and 2 in the swapping condition in the next steps. 

**Step 3:** We choose the student that we need to swap in the non-school-diverse group using the `Pick_out_the_student` function (Function **4.1.4**). We save the student's information and his/her index in the non-school-diverse group. If all groups in the list are diverse, the `Pick_out_the_student` function will be working with an empty list, and it will return `False`. 

We can break out of the `while` loop in step 3 if all groups are school-diverse (`break` condition 1).

**Step 4:** We run through the list of allocated groups again to find our swapping student. There are **3** smaller steps in this step.

- First, we set up a boolean value to mark whether if we can swap the students or not (`swapped_yet_or_not`) and assign it as `False`. 
- Second, we go through all the other groups (except the non-school-diverse group we pick before) using a `for` loop. For every group we go through we take its index and save the group's information to call the `Check_School_types_in_subgroup` function. This is also to check conditions 1 and 2 in the swapping condition in the next steps. 
- Third, we go through every student in every group by using a second `for` loop and take the student's index in his/her group. Here, we will check using the conditions we stated at the beginning of this part. If we find the student that satisfies all of our conditions, we swap them using the `Group_swapper` function (Function **4.1.6**). After swapping, we switch `swapped_yet_or_not` into `True` and break of of the two `for` loops.

&nbsp;&nbsp;&nbsp;&nbsp;In case we cannot sort anymore since we cannot find a student that can be swapped, this can happen if the tutorial groups have more students coming &nbsp;&nbsp;&nbsp;&nbsp;from the same school than the number of groups, then we break out of the `while` loop (`break` condition 2).


This is the **pseudocode** and **flowchart** of this part:

**Pseudocode:**

`FUNCTION FixSchoolDiversity(groups):` \
    `LOOP FOREVER:`

        undesired_index = -1
        undesired_group = None

        # 1. Find first group that fails school diversity
        For each group index:
            If IsSchoolDiverse(groups[index]) == False:
                undesired_index = index
                undesired_group = groups[index]
                BREAK

        # No undesired groups left → done
        If undesired_group is None:
            BREAK

        # 2. Pick a student from a repeated school
        pick = PickStudentFromRepeatedSchool(undesired_group)
        If pick is False:
            BREAK
        undesired_student_index, undesired_student = pick

        undesired_school_counts = CountSchoolTypes(undesired_group)

        swapped = False

        # 3. Try to find a valid swap partner
        For each group B index ≠ undesired_index:
            swap_group = groups[B]
            swap_school_counts = CountSchoolTypes(swap_group)

            For each student in swap_group:
                If undesired_student.school not in swap_school_counts:
                    If student.school not in undesired_school_counts:
                        If student.gender == undesired_student.gender AND
                           CGPAWithinLimit(undesired_student, student):
                                SwapStudents(groups, undesired_index, undesired_student_index, swap_index, swap_student_index)
                                swapped = True
                                BREAK inner loop

            If swapped:
                BREAK outer loop

        # 4. If no swap is possible → stop
        If swapped == False:
            BREAK

    RETURN groups

----------------------------------------------------------------------------------------------------------------
**Flowchart:**
 #Fix!!

----------------------------------------------------------------------------------------------------------------



The code below is the **Python code** for part **4.2.**:

In [502]:
#Function to keep swapping students to enhance school diversity.
def Swapping_students(list_of_lists):
    while True:

        undesired_group_index = -1 #Step 1
        undesired_group = []

        #Code to find the first non-school-diverse group (Step 2)
        for group_index in range(len(list_of_lists)):
            if not Check_School_Diversity(list_of_lists[group_index]):
                undesired_group_index = group_index
                undesired_group = list_of_lists[group_index]
                break
        #Code to get the school types in that undesired group
        School_types_in_undesired_group = Check_School_types_in_subgroup(undesired_group)
        
        #Code to get our student in the undesired group. 
        if Pick_out_the_student(undesired_group) != False:
            undesired_student_index, undesired_student = Pick_out_the_student(undesired_group) #(Step 3)
        else: # If there is no more undesired_groups, we break out of the while loop (break condition 1)
            break 
        #_____________________________________________________________________#

        #Code to rearrange students as specified in Step 4.

        swapped_yet_or_not = False #Marker to check if we have made a swapping move or not (First step in Step 4)
        
        for swapper_group_index in range(len(list_of_lists)):
            if swapper_group_index == undesired_group_index:
                continue

            #Get the index and school types for the swapper_group (Second step in Step 4)
            swapper_group = list_of_lists[swapper_group_index]
            School_types_in_swapper_group = Check_School_types_in_subgroup(swapper_group)

            for swapper_student_index in range(len(swapper_group)): #(Third step in Step 4)
                swapper_student = swapper_group[swapper_student_index]
                
                if undesired_student["school"] not in School_types_in_swapper_group.keys():
                    #If the undesired student has different school to the swapper group
                    
                    if swapper_student["school"] not in School_types_in_undesired_group.keys(): 
                        #If the swapper student has different school to the undesired group
                        
                        if swapper_student["gender"] == undesired_student["gender"] and CGPA_variance_check(undesired_student, swapper_student) == True:
                            #If the swapper student has the same gender and CGPA around as the undesired student(5%)
                                
                            list_of_lists = Group_swapper(list_of_lists, undesired_group_index, undesired_student_index, swapper_group_index, swapper_student_index)
                            #After satisfying all the conditions above, we swap students 
                            
                            swapped_yet_or_not = True #Mark as having swapped the 2 students
                            break #Get out of the small loop

            if swapped_yet_or_not == True: #After swapping, of course get out of the big loop
                break
        
        if swapped_yet_or_not == False: 
            #In case when the number of students with the same school is larger than the number of groups (break condition 2)
            break
    
    return list_of_lists

### **5. Write the output dictionary as a `.csv` file**
After creating functions to allocate and redistribute students within a tutorial group, we will now apply the functions we created in the previous 2 parts for every single tutorial group to return the students - assigned and marked with their group - and write our output file.

This part will have **2** main parts: 
+ Part 5.1: Generate our allocated groups within the tutorial group.
+ Part 5.2: Write the output data down.

#### **5.1. Generate the students allocated files:**
We made a function with the single parameter being the data we extracted in Part 1. The function consists of **4** steps:
1. We created an empty dictionary to add `key:value` pairs, with keys as the tutorial group name and values as the tutorial groups that has assgined their students with their respective mini-groups.
2. We run a `for` loop through all the `key:value` pairs of the input data, and allocate the students in the tutorial group student list associated with the `key` (Tutorial group name) by calling `Groups_of_5` (Function in part `3`) and `Swapping_students` (Function in part `4.2`) to allocate students into their mini groups.
3. After assigning students into their subgroups, we run a `for` loop through all students to mark them. We mark them by adding a `key:value` pair into the student's dictionary with the key is `"team_assigned"` and values is n the format: `f"{students["tutorial_group"]}-T{i + 1}"`, with `students["tutorial_group"]` the tutorial group's name and `i` the index of the subgroup in the tutorial group.
4. After adding every student in the tutorial group with their subgroup's information, we add the tutorial group into the dictionary we made in Step 1


In [503]:
def Group_Sorting(groups):

    Output_groups = {}
    #The output should be a dict with keys are the original groups and values are the lists of lists of allocated groups

    for tut_group_name, student_list in groups.items():
        #Code to use our Groups_of_5 and Swapping_students functions:
        subgroups = Groups_of_5(student_list)
        subgroups = Swapping_students(subgroups)

        #To mark the subgroups with its index:
        for i, subgroup in enumerate(subgroups):
            for students in subgroup:
                students["team_assigned"] = f"{students["tutorial_group"]}-T{i + 1}"

        Output_groups[tut_group_name] = subgroups
    
    return Output_groups

#### **Part 5.2: Write the data out as a `.csv` file:**

We created a function to return our group allocations in a `.csv` file based on the output dictionary after allocating the students in their respective groups. The function consists of **3** steps:
1. We write the first line of the file, which is: "Tutorial Group,Student ID,School,Name,Gender,CGPA,Team Assigned"
2. We pick each tutorial group by ascending order from `"G-1"` to `"G-120"` by using a `for` loop running from 0 to 119 to call each `key:value` pairs.
3. In each iteration of the 2 outer `for` loops, we write down in each line the information of every single student and their assigned groups.

In [504]:
#Function to write our output .csv file.
def WriteOutput(Output_groups):
    with open("OutputFile.csv", mode='w') as file:
        file.write("Tutorial Group,Student ID,School,Name,Gender,CGPA,Team Assigned" + "\n")
        for i in range(120): 
            for allocated_groups in Output_groups[f"G-{i + 1}"]: 
                for students in allocated_groups: 
                    row_values =[
                                students["tutorial_group"],
                                students["student_id"],
                                students["school"],
                                students["name"],
                                students["gender"],
                                str(students["cgpa"]),
                                students["team_assigned"]
                                ]
                    data_row = ",".join(row_values) + "\n"
                    file.write(data_row)



### **6. Analyse our output `.csv` file**

We will now read the output file to analyse the effectiveness of our algorithm.

In [505]:
import csv
import matplotlib.pyplot as plt

In [506]:
# --- CONFIGURATION ---
INPUT_FILE = 'OutputFile.csv'
REQUIRED_TEAM_SIZE = 5

# --- DATA STRUCTURES ---

# Stores the full nested student data grouped by Tutorial Group and then Team
# Structure: { 'G-1': { 'G-1-T1': [{data}, ...], ... }, ... }
tut_groups_map = {}

# Stores results for plotting and summarizing
group_stats = {}

# Stores the final counts for bar charts
gender_ratio_counts = {'5:0': 0, '4:1': 0, '3:2': 0, '2:3': 0, '1:4': 0, '0:5': 0, 'Other': 0}
school_diversity_counts = {5: 0, 4: 0, 3: 0, 2: 0, 1: 0} # Key is number of unique schools

# Stores the standard deviation distribution
sd_distribution_counts = {}

# Define the boundaries for the 0.01 interval bins (up to 0.15)
SD_BINS = [round(i * 0.01, 2) for i in range(17)] # 0.00, 0.01, ..., 0.16

Challenges and issues.
- Find ways to tackle this: Trees at first, but balancing is hard. #Can put at the beginning to showcase our thinking process.

- Consider different gender ratios
- More students from same school
- CGPA means is fine
- Organising appropriate DS to contain students.
- Enhanced algorithm: Do the math to find gender ratios (If have time to write)
- A bit hard to satisfy the conditions to swap.
- GPA not really balanced out.
- Generalise the algo to hard.